# 几何畸变校正探索

In [1]:
import ee 
import geemap
import numpy as np
import pandas as pd
import eerepr
import folium
import copy
import math
from tqdm import trange,tqdm
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

D:\Wrz\anaconda\envs\GEE\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Enter verification code:  4/1AbUR2VOigOuB_gfShqvE7ZmugKgDbieRFHoCsIN0zgbIcB2bYb1iQkHwFc8



Successfully saved authorization token.
geemap version = 0.20.0
geemap path = ['D:\\Wrz\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


In [2]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))

# 计算geometry、质心点、最小包络矩形 
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})
Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())

Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [5]:
def clip_AOI(col, AOI):
    return col.clip(AOI)

def time_difference(col, middle_date):
    time_difference = middle_date.difference(
        ee.Date(col.get('system:time_start')), 'days').abs()
    return col.set({'time_difference': time_difference})

def rm_nodata(col, AOI):
    # 将遮罩外的元素置-99,默认的遮罩为Nodata区域，并统计Nodata的数量
    allNone_num = col.select('VV_sigma0').unmask(-99).eq(-99).reduceRegion(
        **{
            'geometry': AOI,
            'reducer': ee.Reducer.sum(),
            'scale': 10,
            'maxPixels': 1e12,
            'bestEffort': True,
        }).get('VV_sigma0')
    return col.set({'numNodata': allNone_num})

def load_image_collection(aoi, start_date, end_date, middle_date,
                         dem=None,model=None,buffer=0):
    '''
    s1数据加载
    '''
    s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
              .filter(ee.Filter.eq('instrumentMode', 'IW'))
                .filterBounds(aoi)
                .filterDate(start_date, end_date))

    # 地形矫正，可选
    if dem and model:
        print('Begin Slop Correction ...')
        s1_col = slope_correction(s1_col, dem, model, buffer=buffer)
    else:
        Rename = lambda image:image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
        s1_col = s1_col.map(Rename)
        print('Without Slop Correction')
        
    # 裁剪并计算空洞数量
    s1_col = s1_col.map(partial(clip_AOI, AOI=aoi))
    s1_col = s1_col.map(partial(rm_nodata, AOI=aoi))

    # 计算与目标日期的差距
    s1_col = s1_col.map(partial(time_difference, middle_date=middle_date))
    
    # 删除角度波段select
    if model:
        pass
    else:
        s1_col = s1_col.map(lambda img: img.select(['VV_sigma0', 'VH_sigma0']))
      
    # 分离升降轨
    s1_descending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'DESCENDING'))
    s1_ascending = s1_col.filter(ee.Filter.eq(
        'orbitProperties_pass', 'ASCENDING'))
    
    # 如果所有图像都包含空洞则返回一张合成的图像，还需要重投影，否则计算会出错
    if s1_descending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
        s1_descending = s1_descending.median().reproject(s1_descending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
        s1_descending = s1_descending.filter(ee.Filter.eq('numNodata', 0))

    if s1_ascending.filter(ee.Filter.eq('numNodata', 0)).size().getInfo() == 0:
        s1_ascending = s1_ascending.median().reproject(s1_ascending.first().projection().getInfo()['crs'], None, 10).clip(aoi)
    else:
        s1_ascending = s1_ascending.filter(ee.Filter.eq('numNodata', 0))

    return s1_ascending, s1_descending

def Geemap_export(filename,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                        out_dir=os.path.dirname(out_dir),
                        format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:     
        geemap.ee_export_image(image,
                    filename=filename,
                    scale=scale, region=region, file_per_band=False,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

In [13]:
# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
START_DATE  = ee.Date('2019-07-01')
END_DATE   = ee.Date('2019-08-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')
dem = ee.Image('USGS/SRTMGL1_003') # paths to dem 
models = ['volume', 'surface', None]     # 地形矫正模型

# 选择一个点做实验
i=1 #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题)

AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

# 计算面积
AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()
# 缩小
if AOI_area_area < 1: 
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)
    
# 扩大包络矩形AOI,保证背景像素占比最大
if AOI_area_area < 1: 
    AOI_buffer = AOI.buffer(distance=300)
else:
    AOI_buffer = AOI.buffer(distance=400)

In [ ]:
#-----------------------------------------------地形校正-----------------------------------------
def slope_correction(collection, elevation, model, buffer=0):
    '''This function applies the slope correction on a collection of Sentinel-1 data
       
       :param collection: ee.Collection or ee.Image of Sentinel-1
       :param elevation: ee.Image of DEM
       :param model: model to be applied (volume/surface)
       :param buffer: buffer in meters for layover/shadow amsk
        
       :returns: ee.Image
    '''
    def _volumetric_model_SCF(theta_iRad, alpha_rRad):
        '''Code for calculation of volumetric model SCF
            体积模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        
        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
        denominator = (ninetyRad.subtract(theta_iRad)).tan()
        return nominator.divide(denominator)

    def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
        '''Code for calculation of direct model SCF
            表面模型
        :param theta_iRad: ee.Image of incidence angle in radians
        :param alpha_rRad: ee.Image of slope steepness in range
        :param alpha_azRad: ee.Image of slope steepness in azimuth
        
        :returns: ee.Image
        '''

        # create a 90 degree image in radians
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

        # model
        nominator = (ninetyRad.subtract(theta_iRad)).cos()
        denominator = (alpha_azRad.cos().multiply(
            (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

        return nominator.divide(denominator)

    def _erode(image, distance):
        '''Buffer function for raster
            腐蚀算法，输入的图像需要额外的缓冲
        :param image: ee.Image that shoudl be buffered
        :param distance: distance of buffer in meters

        :returns: ee.Image
        '''

        d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt().multiply(
            ee.Image.pixelArea().sqrt()))

        return image.updateMask(d.gt(distance))

    def _masking(alpha_rRad, theta_iRad, buffer):
        '''Masking of layover and shadow
            获取几何畸变区域
        :param alpha_rRad: ee.Image of slope steepness in range
        :param theta_iRad: ee.Image of incidence angle in radians
        :param buffer: buffer in meters
        
        :returns: ee.Image
        '''
        # layover, where slope > radar viewing angle
        layover = alpha_rRad.lt(theta_iRad).rename('layover')

        # shadow
        ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
        shadow = alpha_rRad.gt(
            ee.Image.constant(-1).multiply(
                ninetyRad.subtract(theta_iRad))).rename('shadow')

        # add buffer to layover and shadow
        if buffer > 0:
            layover = _erode(layover, buffer)
            shadow = _erode(shadow, buffer)

        # combine layover and shadow
        no_data_mask = layover.And(shadow).rename('no_data_mask')

        return layover.addBands(shadow).addBands(no_data_mask)

    def _correct(image):
        '''
        This function applies the slope correction and adds layover and shadow masks
        '''

        # get the image geometry and projection
        geom = image.geometry()
        proj = image.select(1).projection()

        # 计算雷达视线向，ee.Terrain.aspect()本是用来计算坡度，该函数返回每个像素点相对于正北方向的坡度角度值
        heading = (ee.Terrain.aspect(image.select('angle')).reduceRegion(
            ee.Reducer.mean(), geom, 1000).get('aspect'))

        # Sigma0 to Power of input image，Sigma0是指雷达回波信号的强度
        sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

        # the numbering follows the article chapters
        # 2.1.1 Radar geometry
        theta_iRad = image.select('angle').multiply(np.pi / 180)
        phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180)

        # 2.1.2 Terrain geometry
        alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)
        phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
            np.pi / 180).setDefaultProjection(proj).clip(geom)

        # we get the height, for export
        height = elevation.setDefaultProjection(proj).clip(geom)

        # 2.1.3 Model geometry
        #reduce to 3 angle
        phi_rRad = phi_iRad.subtract(phi_sRad)

        # slope steepness in range (eq. 2)
        alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

        # slope steepness in azimuth (eq 3)
        alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

        # local incidence angle (eq. 4)
        theta_liaRad = (alpha_azRad.cos().multiply(
            (theta_iRad.subtract(alpha_rRad)).cos())).acos()
        theta_liaDeg = theta_liaRad.multiply(180 / np.pi)

        # 2.2
        # Gamma_nought
        gamma0 = sigma0Pow.divide(theta_iRad.cos())
        gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(
            ['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
        ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(
            gamma0dB.select('VH_gamma0')).rename('ratio_gamma0'))

        if model == 'volume':
            scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

        if model == 'surface':
            scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

        # apply model for Gamm0_f
        gamma0_flat = gamma0.divide(scf)
        gamma0_flatDB = (ee.Image.constant(10).multiply(
            gamma0_flat.log10()).select(['VV', 'VH'],
                                        ['VV_gamma0flat', 'VH_gamma0flat']))

        masks = _masking(alpha_rRad, theta_iRad, buffer)

        # calculate the ratio for RGB vis
        ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
            gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))

        return (image.rename(['VV_sigma0', 'VH_sigma0','incAngle'])
                .addBands(gamma0dB)
                .addBands(ratio_gamma)
                .addBands(gamma0_flatDB)
                .addBands(ratio_flat)
                .addBands(alpha_rRad.rename('alpha_rRad'))
                .addBands(alpha_azRad.rename('alpha_azRad'))
                .addBands(phi_sRad.rename('aspect'))
                .addBands(alpha_sRad.rename('slope'))
                .addBands(theta_iRad.rename('theta_iRad'))
                .addBands(theta_liaRad.rename('theta_liaRad'))
                .addBands(masks).addBands(height.rename('elevation')))

    # run and return correction
    if type(collection) == ee.imagecollection.ImageCollection:
        return collection.map(_correct)
    elif type(collection) == ee.image.Image:
        return _correct(collection)
    else:
        print('Check input type, only image collection or image can be input')
        



# 在这个点采用buffer裁剪文件，展示SAR，去除了包含空值的影像
s1_ascending,s1_descending = load_image_collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE,
                          dem=dem,model=models[1])

# # 全包含空值条件下，将图像集合合并为一张图像进行处理
# if type(s1_ascending) == ee.imagecollection.ImageCollection:
#     s1_single_a = s1_ascending.filter(ee.Filter.eq('time_difference', 
#           s1_ascending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
# else:
#     s1_single_a = s1_ascending.set({'synthesis': 1})
# if type(s1_descending) == ee.imagecollection.ImageCollection:
#     s1_single_d = s1_descending.filter(ee.Filter.eq('time_difference', 
#           s1_descending.aggregate_min('time_difference'))).first().set({'synthesis': 0})
# else:
#     s1_single_d = s1_descending.set({'synthesis': 1})

# # 加入条件空值，保证在空值情况下图像能够正常相加
# condition = s1_single_d.mask().clip(AOI_buffer)
# s1_unit_mean = s1_single_a.where(condition, s1_single_a.add(s1_single_d).divide(2)) #转为均值
# s1_unit_max = s1_single_a.where(condition, s1_single_a.max(s1_single_d))
# # s1_unit_add = s1_single_a.where(condition, s1_single_a.add(s1_single_d))

# s1_unit_all = s1_unit_mean.addBands(s1_unit_max)


Begin Slop Correction ...
